In [1]:
import tensorflow as tf
from tensorflow_core.examples.tutorials.mnist import input_data
import numpy as np
class MNISTLoader_my_download():
    def __init__(self):
        # 读取数据，预先已经下载了相应的数据直
        mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
        self.train_data = mnist.train.images
        self.train_label = mnist.train.labels
        self.test_data = mnist.test.images
        self.test_label = mnist.test.labels
        
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 784, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 784, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

In [2]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)  # 第一层神经元的个数为100
        self.dense2 = tf.keras.layers.Dense(units=10)   # 第二层神经元的个数为10,输出一个样本的维度为10

    @tf.function
    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [5]:
# 定义一些模型超参数：
num_epochs = 5
batch_size = 50
learning_rate = 0.001

# 实例化模型和数据读取类，并实例化一个 tf.keras.optimizer 的优化器（这里使用常用的 Adam 优化器）：
model = MLP()
# data_loader = MNISTLoader() # 导入数据 
data_loader = MNISTLoader_my_download()  # 导入数据
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # 更新梯度

summary_writer = tf.summary.create_file_writer('./tensorboard')
tf.summary.trace_on(graph=True, profiler=True)  # 开启Trace，可以记录图结构和profile信息

# num_batches = int(mnist.train.num_examples // batch_size * num_epochs)
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(5):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))

    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    

# 进行训练
with summary_writer.as_default():
    tf.summary.trace_export(name="model_trace", step=0, profiler_outdir='./tensorboard')    # 保存Trace信息到文件
    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
batch 0: loss 2.302626
batch 1: loss 2.301785
batch 2: loss 2.302305
batch 3: loss 2.302342
batch 4: loss 2.300716


In [11]:
categorical_accuracy = tf.keras.metrics.CategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % categorical_accuracy.result())

test accuracy: 0.106100
